In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from asbe.base import *
from asbe.models import *
from asbe.estimators import *
from dataclasses import dataclass
import numpy as np
from collections.abc import Callable
from typing import Union

In [ ]:
# Data generating process

In [ ]:
@dataclass
class DataGenerator():
    ds: str = None
    dgp_x : Union[Callable, None] = None
    
    def get_X(self, method : str = "random", no_query : int = 1):
        if self.ds is not None:
            ix = np.random.randint(low = 0, high = self.ds["X_pool"].shape[0]-1, size=(no_query))
            X_new = self.ds["X_pool"][ix,:]
        else:
            X_new = self.dgp_x()

In [ ]:
dg = DataGenerator()

In [ ]:
dg.get_X()

In [ ]:
dg.__dict__

{'ds': None, 'dgp_x': None}